# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import json
import datetime
now = datetime.datetime.now()
print(now.replace(second=0, microsecond=0))

2024-06-22 10:56:00


In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

# Start the database server using a command line command.  It is not recommended to start from Jypyter notebooks.
### This starts the database server (just a reminder to me as I sometimes forget how to start the MongoDB.
* mongod

In [3]:
# confirm that our new database was created
db_names = mongo.list_database_names()
print("Existing databases:")
print(db_names)

Existing databases:
['ClassDB2', 'admin', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'testing', 'travel_db']


In [4]:
# assign the uk_food database to a variable name
uk_db = 'uk_food'
# Need to drop to make sure that duplicate values are not inserted if the database and collection exists
db = mongo[uk_db]
db.establishments.drop()  # Replace with your collection name

In [5]:
# Specify the file path
file_path = './Resources/establishments.json'

# Load JSON data from the file with UTF-8 encoding
with open(file_path, 'r', encoding='utf-8') as file:
    establishments_data = json.load(file)

# Create a collection and insert the data
collection = db['establishments']
collection.insert_many(establishments_data)

print("Data imported and collection 'establishments' created.")

Data imported and collection 'establishments' created.


In [6]:
# review the collections in our new database
collection_names = db.list_collection_names()

# Print the collection names
print("Collections in the 'uk_food' database:")
for name in collection_names:
    print(name)

Collections in the 'uk_food' database:
establishments


In [7]:
# review a document in the establishments collection
# Specify the collection name
collection_name = 'establishments'

# Find one document in the collection
document = db[collection_name].find_one()

# Print the document
print("Document in the 'establishments' collection:")
print(document)

Document in the 'establishments' collection:
{'_id': ObjectId('66771044bd4e37ace307cb18'), 'FHRSID': 254719, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000069980', 'BusinessName': 'Refreshment Kiosk', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'The Bay', 'AddressLine2': 'St Margarets Bay', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT15 6DY', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2022-03-24T00:00:00', 'LocalAuthorityCode': '182', 'LocalAuthorityName': 'Dover', 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/', 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'}, 'RightToReply': '', 'Distance': 4587.347174863443, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0

In [8]:
# assign the collection to a variable
collection_name = 'establishments'

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "name": "New Restaurant",
    "address": "123 Main St",
    "city": "London",
    "postcode": "SW1A 1AA",
    "cuisine": "Italian",
    "rating": 4.5
}

In [10]:
# Insert the new restaurant into the collection
result = db[collection_name].insert_one(new_restaurant_data)

# Print the inserted document's ID
print("Inserted document ID:", result.inserted_id)
inserted_document_id = result.inserted_id

Inserted document ID: 66771048bd4e37ace308667b


In [11]:
# Check that the new restaurant was inserted
query = {"name": "New Restaurant"}
restaurant_document = db[collection_name].find_one(query)

# Check if the document exists
if restaurant_document:
    print("New restaurant found:")
    print(restaurant_document)
else:
    print("New restaurant not found.")

New restaurant found:
{'_id': ObjectId('66771048bd4e37ace308667b'), 'name': 'New Restaurant', 'address': '123 Main St', 'city': 'London', 'postcode': 'SW1A 1AA', 'cuisine': 'Italian', 'rating': 4.5}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

# Define the query
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Define the projection to return only the BusinessTypeID and BusinessType fields
projection = {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}

# Find the document matching the query with projection
result = db[collection_name].find_one(query, projection)

# Print the result
print(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
from bson import ObjectId

establishments = db.establishments  

# Specify the ObjectId of the document to update
document_id = ObjectId(inserted_document_id)

# Define the new field to add/update
new_field = {"BusinessTypeID": 1}

# Update the document with the new field
establishments.update_one(
    {"_id": document_id},  # Filter by document ID
    {"$set": new_field}  # Set the new field
)

# Optionally, retrieve the updated document to confirm the change
updated_document = establishments.find_one({"_id": document_id})
print(updated_document)

{'_id': ObjectId('66771048bd4e37ace308667b'), 'name': 'New Restaurant', 'address': '123 Main St', 'city': 'London', 'postcode': 'SW1A 1AA', 'cuisine': 'Italian', 'rating': 4.5, 'BusinessTypeID': 1}


In [14]:
# Confirm that the new restaurant was updated
# Find the document by its ObjectId
updated_document = collection.find_one({"_id": document_id})

# Check the document to confirm the update
print("Updated document:")
print(updated_document)

Updated document:
{'_id': ObjectId('66771048bd4e37ace308667b'), 'name': 'New Restaurant', 'address': '123 Main St', 'city': 'London', 'postcode': 'SW1A 1AA', 'cuisine': 'Italian', 'rating': 4.5, 'BusinessTypeID': 1}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

# Count the documents matching the query
count = db[collection_name].count_documents(query)

# Print the count
print("Number of documents with LocalAuthorityName as 'Dover':", count)

Number of documents with LocalAuthorityName as 'Dover': 994


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}
result = db[collection_name].delete_many(query)
print("Number of documents deleted:", result.deleted_count)

Number of documents deleted: 994


In [17]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}

# Count the number of remaining documents matching the query
count_remaining = db[collection_name].count_documents(query)

# Check if any documents are found
if count_remaining > 0:
    print("There are remaining documents with LocalAuthorityName as 'Dover'.")
else:
    print("No remaining documents with LocalAuthorityName as 'Dover'.")

# Optionally, print the count of remaining documents
print("Number of remaining documents:", count_remaining)

No remaining documents with LocalAuthorityName as 'Dover'.
Number of remaining documents: 0


In [18]:
# Check that other documents remain with 'find_one'
remaining_document = db[collection_name].find_one()
if remaining_document:
    print("At least one document remains in the collection.")
    print("Example remaining document:", remaining_document)
else:
    print("No documents remain in the collection.")

At least one document remains in the collection.
Example remaining document: {'_id': ObjectId('66771044bd4e37ace307cdff'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta'

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Set non 1-5 Rating Values to Null
documents = collection.find()

# Update the documents
for document in documents:
    # Check if 'RatingValue' exists and is of type string
    if 'RatingValue' in document:
        try:
            # Convert the RatingValue to integer if it's a string
            if isinstance(document['RatingValue'], str):
                rating_value_int = int(document['RatingValue'])
            else:
                rating_value_int = document['RatingValue']

            # Check if the rating value is between 1 and 5
            if 1 <= rating_value_int <= 5:
                # Update the document with the valid integer value
                collection.update_one(
                    {'_id': document['_id']},
                    {'$set': {'RatingValue': rating_value_int}}
                )
            else:
                # Set the RatingValue to null if it's not between 1 and 5
                collection.update_one(
                    {'_id': document['_id']},
                    {'$set': {'RatingValue': None}}
                )
        except ValueError:
            # Handle the case where conversion to integer fails
            # Set the RatingValue to null
            collection.update_one(
                {'_id': document['_id']},
                {'$set': {'RatingValue': None}}
            )
    else:
        # Set the RatingValue to null if it doesn't exist
        collection.update_one(
            {'_id': document['_id']},
            {'$set': {'RatingValue': None}}
        )

# Check the updated documents (optional)
for document in collection.find():
    print(document)

{'_id': ObjectId('66771044bd4e37ace307cdff'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': 5, 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'r

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



{'_id': ObjectId('66771045bd4e37ace308081b'), 'FHRSID': 1426816, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': '21/00223/BAKER', 'BusinessName': 'Flour And Spoon At Artisan Market', 'BusinessType': 'Retailers - other', 'BusinessTypeID': 4613, 'AddressLine1': '198 - 200 Leigh Road', 'AddressLine2': 'Leigh-on-sea', 'AddressLine3': 'Essex', 'AddressLine4': '', 'PostCode': 'SS9 1BS', 'Phone': '', 'RatingValue': 4, 'RatingKey': 'fhrs_4_en-gb', 'RatingDate': '2021-11-26T00:00:00', 'LocalAuthorityCode': '893', 'LocalAuthorityName': 'Southend-On-Sea', 'LocalAuthorityWebSite': 'http://www.southend.gov.uk', 'LocalAuthorityEmailAddress': 'EnvironmentalHealth@southend.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.6649304', 'latitude': '51.5430147'}, 'RightToReply': '', 'Distance': 4628.2082054407, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, '

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




{'_id': ObjectId('66771045bd4e37ace30860d1'), 'FHRSID': 872556, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'COMM/09/16903', 'BusinessName': 'Church Langley Church', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': 'Church Langley Community Centre', 'AddressLine2': 'Church Langley Way', 'AddressLine3': 'Harlow', 'AddressLine4': 'Essex', 'PostCode': 'CM17 9TG', 'Phone': '', 'RatingValue': 5, 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2011-01-27T00:00:00', 'LocalAuthorityCode': '116', 'LocalAuthorityName': 'Harlow', 'LocalAuthorityWebSite': 'http://www.harlow.gov.uk/', 'LocalAuthorityEmailAddress': 'env.health@harlow.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.13275', 'latitude': '51.767732'}, 'RightToReply': '', 'Distance': 4655.7881638317585, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode'

In [20]:
import decimal
collection = db[collection_name]

documents = collection.find({
    "$or": [
        {"geocode.longitude": {"$type": "string"}},#Find type string
        {"geocode.latitude": {"$type": "string"}}#Find type string
    ]
})

# Any ducument with type string above
for document in documents: #Must have geocode
    if 'geocode' in document:
        geocode = document['geocode']
        
        longitude_str = geocode.get('longitude', None)
        latitude_str = geocode.get('latitude', None)

        if longitude_str and latitude_str:
            try:
                longitude_float = float(longitude_str)# Try to convert to float
                latitude_float = float(latitude_str)# Try to convert to float

                # Try to update the document
                collection.update_one(
                    {'_id': document['_id']},
                    {'$set': {
                        'geocode.longitude': longitude_float,
                        'geocode.latitude': latitude_float
                    }}
                )
            except ValueError:
                pass #If there is an error do not process this record.

In [21]:
# Use `update_many` to convert `RatingValue` to integer numbers
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [22]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['your_database_name']
collection = db['your_collection_name']

# Retrieve the documents
documents = collection.find()

# Update the documents
for document in documents:
    # Check if 'RatingValue' exists and is of type string
    if 'RatingValue' in document and isinstance(document['RatingValue'], str):
        try:
            # Convert the RatingValue to integer
            rating_value_int = int(document['RatingValue'])

            # Update the document
            collection.update_one(
                {'_id': document['_id']},
                {'$set': {'RatingValue': rating_value_int}}
            )
        except ValueError:
            # Handle the case where conversion to integer fails
            print(f"Failed to convert RatingValue to integer for document with _id: {document['_id']}")

# Check the updated documents (optional)
for document in collection.find():
    print(document)


In [23]:
# Change the data type from String to Integer for RatingValue
sample_document = collection.find_one()  # You can also retrieve multiple documents to check consistency

# Ensure the sample document is not None
for document in documents:
    # Initialize variables for data types
    latitude_type = None
    longitude_type = None
    rating_value_type = None

    # Check if 'geocode' and 'RatingValue' fields exist
    if 'geocode' in sample_document:
        latitude_type = type(sample_document['geocode'].get('latitude', None))
        longitude_type = type(sample_document['geocode'].get('longitude', None))
    
    if 'RatingValue' in sample_document:
        rating_value_type = type(sample_document['RatingValue'])

    # Display the data types
    #print("Latitude data type:", latitude_type)
    #print("Longitude data type:", longitude_type)
    #print("RatingValue data type:", rating_value_type)
#else:
#    print("No document found in the collection.")

In [24]:
documents = collection.find()

# Iterate over the documents
for document in documents:
    # Check the data type of RatingValue
    rating_value = document.get('RatingValue', None)
    longitude = document.get('geocode', {}).get('longitude', None)
    latitude = document.get('geocode', {}).get('latitude', None)
    
    # Check if RatingValue is a number
    if isinstance(rating_value, int):
        print(f"Document {document['_id']} has RatingValue as an integer: {rating_value}")
    elif isinstance(rating_value, str) and rating_value.isdigit():
        print(f"Document {document['_id']} has RatingValue as a numeric string: {rating_value}")
    else:
        print(f"Document {document['_id']} has non-numeric RatingValue: {rating_value}")

    # Check if the coordinates are numbers
    if isinstance(longitude, (int, float)) and isinstance(latitude, (int, float)):
        print(f"Document {document['_id']} has numeric coordinates: longitude={longitude}, latitude={latitude}")
    else:
        print(f"Document {document['_id']} has non-numeric coordinates: longitude={longitude}, latitude={latitude}")


In [25]:
collection = db['establishments']  # Ensure this is correct

# Debug: Check if the collection has any documents
document_count = collection.count_documents({})
print(f"Total documents in 'establishments': {document_count}")

if document_count == 0:
    print("No documents found in the 'establishments' collection.")
else:
    # Find all documents in the collection
    documents = collection.find()

    # Iterate over the documents
    for document in documents:
        # Check the data type of RatingValue
        rating_value = document.get('RatingValue', None)
        longitude = document.get('geocode', {}).get('longitude', None)
        latitude = document.get('geocode', {}).get('latitude', None)
        
        # Check if RatingValue is a number
        if isinstance(rating_value, int):
            print(f"Document {document['_id']} has RatingValue as an integer: {rating_value}")
        elif isinstance(rating_value, str) and rating_value.isdigit():
            print(f"Document {document['_id']} has RatingValue as a numeric string: {rating_value}")
        else:
            print(f"Document {document['_id']} has non-numeric RatingValue: {rating_value}")

        # Check if the coordinates are numbers
        if isinstance(longitude, (int, float)) and isinstance(latitude, (int, float)):
            print(f"Document {document['_id']} has numeric coordinates: longitude={longitude}, latitude={latitude}")
        else:
            print(f"Document {document['_id']} has non-numeric coordinates: longitude={longitude}, latitude={latitude}")


Total documents in 'establishments': 0
No documents found in the 'establishments' collection.


In [26]:
# review a document in the establishments collection
# Specify the collection name
collection_name = 'establishments'

# Find one document in the collection
document = db[collection_name].find_one()

# Print the document
print("Document in the 'establishments' collection:")
print(document)

Document in the 'establishments' collection:
None
